In [ ]:
import asyncio
import numpy as np
import time
from concurrent.futures import ThreadPoolExecutor

# 1. A heavy synchronous Numpy function
def heavy_numpy_math(n):
    print(f"--> Starting heavy math for N={n}")
    start = time.time()
    
    # Create huge matrices
    a = np.random.rand(n, n)
    b = np.random.rand(n, n)
    
    # Heavy matrix multiplication
    result = np.dot(a, b)
    
    end = time.time()
    print(f"<-- Finished math for N={n} in {end - start:.4f}s")
    return result[0, 0]

# 2. The Async Wrapper
async def run_math_async(executor, n):
    loop = asyncio.get_running_loop()
    
    # This is the magic line. It runs the blocking function 
    # in a separate thread so the loop stays free.
    result = await loop.run_in_executor(executor, heavy_numpy_math, n)
    return result

# 3. A heartbeat to prove the loop isn't blocked
async def heart_beat():
    while True:
        print("❤️ Loop is alive...")
        await asyncio.sleep(0.5)

async def main():
    # Create a thread pool
    executor = ThreadPoolExecutor(max_workers=3)
    
    # Start the heartbeat (background task)
    heartbeat_task = asyncio.create_task(heart_beat())
    
    print("Main: Starting math tasks...")
    
    # Launch 3 heavy math operations concurrently
    # If we didn't use run_in_executor, the Heartbeat would stop beating!
    await asyncio.gather(
        run_math_async(executor, 2000),
        run_math_async(executor, 3000),
        run_math_async(executor, 2500)
    )
    
    print("Main: All math done.")
    heartbeat_task.cancel()



In [5]:
await main()

Main: Starting math tasks...
❤️ Loop is alive...
--> Starting heavy math for N=2000
--> Starting heavy math for N=3000
--> Starting heavy math for N=2500
<-- Finished math for N=2000 in 0.3809s
❤️ Loop is alive...
<-- Finished math for N=3000 in 0.8911s
❤️ Loop is alive...
<-- Finished math for N=2500 in 1.2241s
Main: All math done.
